In [1]:
import string
import xmltodict
import untangle
import pandas as pd

## Melee

In [55]:
def parse_xml(fp: str) -> pd.DataFrame:
    xml = untangle.parse(fp)
    data = []
    for thingdef in xml.Defs.ThingDef:
        try:
            name = thingdef.defName.cdata
        except AttributeError:
            continue
        data.append(
            {
                "name": name,
                "attacks": [parse_attack(li) for li in thingdef.tools.li]
            }
        )
    df = pd.DataFrame(data)
    df["dps"] = df.attacks.apply(lambda x: calculate_dps(x))
    df["ap"] = [max([attack["armorPenetration"] for attack in attacks]) * 100 for attacks in df.attacks]
    df.set_index("name", inplace=True)
    #df.drop(columns="attacks", inplace=True)
    return df

In [46]:
def parse_attack(li: untangle.Element):
    attack = {}
    attack["damage"] = float(li.power.cdata)
    attack["cooldown"] = float(li.cooldownTime.cdata)
    if li.get_elements("armorPenetration"):
        attack["armorPenetration"] = float(li.armorPenetration.cdata)
    else:
        attack["armorPenetration"] = attack["damage"] * 0.015
    return attack

In [47]:
def calculate_dps(attacks: list) -> float:
    weights = [attack["damage"] * attack["damage"] for attack in attacks]
    chances = [weight / sum(weights) for weight in weights]
    avg_dmg = sum([attacks[i]["damage"] * chances[i] for i in range(len(attacks))])
    avg_cooldown = sum([attacks[i]["cooldown"] * chances[i] for i in range(len(attacks))])
    return round(avg_dmg / avg_cooldown, 2)

In [56]:
medieval = parse_xml("C:/Games/Steam/steamapps/common/RimWorld/Data/Core/Defs/ThingDefs_Misc/Weapons/MeleeMedieval.xml")
neolithic = parse_xml("C:/Games/Steam/steamapps/common/RimWorld/Data/Core/Defs/ThingDefs_Misc/Weapons/MeleeNeolithic.xml")
medieval_ideology = parse_xml("C:/Games/Steam/steamapps/common/RimWorld/Data/Royalty/Defs/ThingDefs_Misc/Weapons/MeleeMedieval.xml")
ultratech = parse_xml("C:/Games/Steam/steamapps/common/RimWorld/Data/Royalty/Defs/ThingDefs_Misc/Weapons/MeleeUltratech.xml")
bladelink = parse_xml("C:/Games/Steam/steamapps/common/RimWorld/Data/Royalty/Defs/ThingDefs_Misc/Weapons/MeleeBladelink.xml")

In [49]:
melee = pd.concat((medieval, neolithic, medieval_ideology, ultratech, bladelink))

In [50]:
melee.sort_values(by="dps", ascending=False)

,dps,ap
name,,
MeleeWeapon_MonoSwordBladelink,16.03,90.00
MeleeWeapon_ZeusHammerBladelink,13.40,46.50
MeleeWeapon_MonoSword,12.08,90.00
MeleeWeapon_PlasmaSwordBladelink,11.11,34.50
MeleeWeapon_Zeushammer,9.95,46.50
MeleeWeapon_LongSword,8.60,34.50
MeleeWeapon_Spear,7.91,50.00
MeleeWeapon_PlasmaSword,7.85,31.50
MeleeWeapon_Gladius,7.52,24.00


In [51]:
vwe_industrial = parse_xml("C:/Games/Steam/steamapps/workshop/content/294100/1814383360/1.3/Defs/ThingDefs_Misc/Weapons/MeleeIndustrial.xml")
vwe_medieval = parse_xml("C:/Games/Steam/steamapps/workshop/content/294100/1814383360/1.3/Defs/ThingDefs_Misc/Weapons/MeleeMedieval.xml")
vwe_neolithic = parse_xml("C:/Games/Steam/steamapps/workshop/content/294100/1814383360/1.3/Defs/ThingDefs_Misc/Weapons/MeleeNeolithic.xml")
vwe_tribal_neolithic = parse_xml("C:/Games/Steam/steamapps/workshop/content/294100/2454918552/1.3/Defs/ThingDefs_Misc/MeleeNeolithic.xml")
vwe_viking_medieval = parse_xml("C:/Games/Steam/steamapps/workshop/content/294100/2231295285/1.3/Defs/ThingDefs_Misc/Weapons/MeleeMedieval.xml")
vwe_viking_ultra = parse_xml("C:/Games/Steam/steamapps/workshop/content/294100/2231295285/1.3/Defs/ThingDefs_Misc/Weapons/MeleeUltra.xml")

In [52]:
melee = pd.concat((melee, vwe_industrial, vwe_medieval, vwe_neolithic, vwe_tribal_neolithic, vwe_viking_medieval, vwe_viking_ultra))

In [60]:
melee.sort_values(by="dps", ascending=False)

,dps,ap
name,,
MeleeWeapon_MonoSwordBladelink,16.03,90.00
MeleeWeapon_ZeusHammerBladelink,13.40,46.50
MeleeWeapon_MonoSword,12.08,90.00
MeleeWeapon_PlasmaSwordBladelink,11.11,34.50
VFEV_CryptoHeavyAxe,10.89,35.00
MeleeWeapon_Zeushammer,9.95,46.50
VWE_MeleeWeapon_Halberd,9.23,70.00
MeleeWeapon_LongSword,8.60,34.50
VWE_MeleeWeapon_BattleAxe,8.43,15.00


* Halberds have too much Armor Penetration
* Combat knifes and Shivs have too high DPS
* Shovels have more DPS than Dane Axe
* Wrench and Seax should be last
* Combat Knife and Shiv have higher AP than Battle Axe and more DPS than Gladius / DaneAxe ?

## Armor

In [57]:
def parse_xml(fp: str) -> pd.DataFrame:
    with open(fp, "rb") as f:
        defs = xmltodict.parse(f)["Defs"]
    data = []
    for thingdef in defs["ThingDef"]:
        name = thingdef.get("defName")
        stats = thingdef.get("statBases")
        offsets = thingdef.get("equippedStatOffsets")
        if name and stats:
            data.append({
                "name": name,
                "stuff_effect_multiplier_armor": stats.get("StuffEffectMultiplierArmor"),
                "armor_rating_sharp": stats.get("ArmorRating_Sharp"),
                "armor_rating_blunt": stats.get("ArmorRating_Blunt"),
                "armor_rating_heat": stats.get("ArmorRating_Heat"),
                "insulation_cold": stats.get("Insulation_Cold"),
                "insulation_heat": stats.get("Insulation_Heat"),
                "stuff_effect_multiplier_insulation_cold": stats.get("StuffEffectMultiplierInsulation_Cold"),
                "stuff_effect_multiplier_insulation_heat": stats.get("StuffEffectMultiplierInsulation_Heat"),
            })
            if offsets:
                data[-1].update(offsets=offsets)
            df = pd.DataFrame(data)
            for col in ("stuff_effect_multiplier_armor", "armor_rating_sharp", "armor_rating_blunt", "armor_rating_heat"):
                df[col] = df[col].apply(lambda x: float(x) if x else pd.NA)
        
    return df

In [63]:
core = parse_xml("C:/Games/Steam/steamapps/common/RimWorld/Data/Core/Defs/ThingDefs_Misc/Apparel_Various.xml")
ideology = parse_xml("C:/Games/Steam/steamapps/common/RimWorld/Data/Ideology/Defs/ThingDefs_Misc/Apparel_Various.xml")
royalty = parse_xml("C:/Games/Steam/steamapps/common/RimWorld/Data/Royalty/Defs/ThingDefs_Misc/Apparel_Various.xml")
vae_industrial = parse_xml("C:/Games/Steam/steamapps/workshop/content/294100/1814988282/1.3/Defs/ThingDefs_Misc/Armor_Industrial.xml")
vae_medieval = parse_xml("C:/Games/Steam/steamapps/workshop/content/294100/1814988282/1.3/Defs/ThingDefs_Misc/Armor_Medieval.xml")
vae_neolithic = parse_xml("C:/Games/Steam/steamapps/workshop/content/294100/1814988282/1.3/Defs/ThingDefs_Misc/Armor_Neolithic.xml")
vae_spacer = parse_xml("C:/Games/Steam/steamapps/workshop/content/294100/1814988282/1.3/Defs/ThingDefs_Misc/Armor_Spacer.xml")
vae_apparel_industrial = parse_xml("C:/Games/Steam/steamapps/workshop/content/294100/1814987817/1.3/Defs/ThingDefs_Misc/Apparel_Industrial.xml")
vae_apparel_medieval = parse_xml("C:/Games/Steam/steamapps/workshop/content/294100/1814987817/1.3/Defs/ThingDefs_Misc/Apparel_Medieval.xml")
vae_apparel_neolithic = parse_xml("C:/Games/Steam/steamapps/workshop/content/294100/1814987817/1.3/Defs/ThingDefs_Misc/Apparel_Neolithic.xml")
vfe_viking = parse_xml("C:/Games/Steam/steamapps/workshop/content/294100/2231295285/1.3/Defs/ThingDefs_Misc/Apparel_Various.xml")

In [66]:
armors = pd.concat((core, ideology, royalty, vae_industrial, vae_medieval, vae_neolithic, vae_spacer, vfe_viking, vae_apparel_industrial, vae_apparel_medieval, vae_apparel_neolithic))
armors["move_speed"] = armors.offsets.apply(lambda x: float(x.get("MoveSpeed", 0)) if not pd.isna(x) else None)

In [68]:
pd.options.display.max_rows = 999

In [95]:
armors.set_index("name").sort_values(by="stuff_effect_multiplier_armor", ascending=False)

,stuff_effect_multiplier_armor,armor_rating_sharp,armor_rating_blunt,armor_rating_heat,insulation_cold,insulation_heat,stuff_effect_multiplier_insulation_cold,stuff_effect_multiplier_insulation_heat,offsets,move_speed
name,,,,,,,,,,
VAE_Apparel_AdvancedVest,1.0,<NA>,<NA>,<NA>,None,None,1,0.6,NaN,NaN
VAE_Apparel_PlateShoulderpads,0.88,<NA>,<NA>,<NA>,None,None,0.1,0,NaN,NaN
VFEV_Apparel_GuardianArmor,0.85,<NA>,<NA>,<NA>,40,9,None,None,"{'MoveSpeed': '-1.1', 'MeleeDodgeChance': '-18'}",-1.10
VAE_Apparel_LightPlateArmor,0.73,<NA>,<NA>,<NA>,None,None,0.14,0.0,{'MoveSpeed': '-0.12'},-0.12
Apparel_PlateArmor,0.73,<NA>,<NA>,<NA>,None,None,1.0,0,{'MoveSpeed': '-0.8'},-0.80
VAE_Apparel_Chestplate,0.73,<NA>,<NA>,<NA>,None,None,0.12,0.12,{'MoveSpeed': '-0.2'},-0.20
VFEV_Apparel_RavagerArmor,0.58,<NA>,<NA>,<NA>,None,None,1.06,0.11,{'PainShockThreshold': '0.10'},0.00
VFEV_LeatherArmor,0.55,<NA>,<NA>,<NA>,None,None,0.90,0.11,NaN,NaN
VAE_Apparel_Chainmail,0.46,<NA>,<NA>,<NA>,None,None,0.05,0.05,{'MoveSpeed': '-0.1'},-0.10


In [16]:
QUALITY = {
    "awful": 0.6,
    "poor": 0.8,
    "normal": 1.0,
    "good": 1.15,
    "excellent": 1.3,
    "masterwork": 1.45,
    "legendary": 1.8
}

STUFF = {
    "metallic": {
        "gold": {"sharp": 0.72, "blunt": 0.36, "heat": 0.36},
        "plasteel": {"sharp": 1.14, "blunt": 0.55, "heat": 0.65},
        "steel": {"sharp": 0.9, "blunt": 0.45, "heat": 0.6},
        "uranium": {"sharp": 1.08, "blunt": 0.54, "heat": 0.65},
        "sky steel": {"sharp": 1.0, "blunt": 0.55, "heat": 0.35},
        "chtin": {"sharp": 0.92, "blunt": 0.18, "heat": 0.27}
    },
    "leather": {
        "thrumbofur": {"sharp": 2.08, "blunt": 0.36, "heat": 1.5},
        "hyperweave": {"sharp": 2, "blunt": 0.54, "heat": 2.88},
        "devilstrand": {"sharp": 1.4, "blunt": 0.36, "heat": 3},
        "heavy fur": {"sharp": 1.24, "blunt": 0.24, "heat": 1.5},
        "bearskin": {"sharp": 1.12, "blunt": 0.24, "heat": 1.5},
        "bluefur": {"sharp": 0.81, "blunt": 0.24, "heat": 1.5}
    }
}
        

def expand(src_rating=0, stuff=None):
    """stuff = metallic | leather | None"""
    data = []
    for quali, quali_factor in QUALITY.items():
        if stuff:
            for material, factors in STUFF[stuff].items():
                data.append({
                    "name": f"{quali}_{material}",
                    "sharp": src_rating * quali_factor * factors["sharp"],
                    "blunt": src_rating * quali_factor * factors["blunt"],
                    "heat": src_rating * quali_factor * factors["heat"]
                })
        else:
            data.append({
                "name": f"{quali}",
                "sharp": src_rating * quali_factor,
                "blunt": src_rating * quali_factor,
                "heat": src_rating * quali_factor
            })
    return pd.DataFrame.from_dict(data)

In [25]:
# leather armor
expand(src_rating=0.45, stuff="leather").sort_values(by="sharp", ascending=False).head(20)

,name,sharp,blunt,heat
36,legendary_thrumbofur,1.6848,0.29160,1.21500
37,legendary_hyperweave,1.6200,0.43740,2.33280
30,masterwork_thrumbofur,1.3572,0.23490,0.97875
31,masterwork_hyperweave,1.3050,0.35235,1.87920
24,excellent_thrumbofur,1.2168,0.21060,0.87750
25,excellent_hyperweave,1.1700,0.31590,1.68480
38,legendary_devilstrand,1.1340,0.29160,2.43000
18,good_thrumbofur,1.0764,0.18630,0.77625
19,good_hyperweave,1.0350,0.27945,1.49040
39,legendary_heavy fur,1.0044,0.19440,1.21500


In [25]:
# leather armor
expand(src_rating=0.45, stuff="leather").sort_values(by="sharp", ascending=False).head(20)

,name,sharp,blunt,heat
36,legendary_thrumbofur,1.6848,0.29160,1.21500
37,legendary_hyperweave,1.6200,0.43740,2.33280
30,masterwork_thrumbofur,1.3572,0.23490,0.97875
31,masterwork_hyperweave,1.3050,0.35235,1.87920
24,excellent_thrumbofur,1.2168,0.21060,0.87750
25,excellent_hyperweave,1.1700,0.31590,1.68480
38,legendary_devilstrand,1.1340,0.29160,2.43000
18,good_thrumbofur,1.0764,0.18630,0.77625
19,good_hyperweave,1.0350,0.27945,1.49040
39,legendary_heavy fur,1.0044,0.19440,1.21500


In [26]:
# advanced vest
expand(src_rating=1., stuff="metallic").sort_values(by="sharp", ascending=False).head(20)

,name,sharp,blunt,heat
37,legendary_plasteel,2.052,0.9900,1.1700
39,legendary_uranium,1.944,0.9720,1.1700
40,legendary_sky steel,1.800,0.9900,0.6300
41,legendary_chtin,1.656,0.3240,0.4860
31,masterwork_plasteel,1.653,0.7975,0.9425
38,legendary_steel,1.620,0.8100,1.0800
33,masterwork_uranium,1.566,0.7830,0.9425
25,excellent_plasteel,1.482,0.7150,0.8450
34,masterwork_sky steel,1.450,0.7975,0.5075
27,excellent_uranium,1.404,0.7020,0.8450


In [52]:
# flak vest
expand(src_rating=0.55, stuff=None).sort_values(by="sharp", ascending=False).head(20)

,name,sharp,blunt,heat
6,legendary,0.9900,0.9900,0.9900
5,masterwork,0.7975,0.7975,0.7975
4,excellent,0.7150,0.7150,0.7150
3,good,0.6325,0.6325,0.6325
2,normal,0.5500,0.5500,0.5500
1,poor,0.4400,0.4400,0.4400
0,awful,0.3300,0.3300,0.3300


In [44]:
# marine armor
expand(src_rating=1.06, stuff=None).sort_values(by="sharp", ascending=False).head(20)

,name,sharp,blunt,heat
6,legendary,1.908,1.908,1.908
5,masterwork,1.537,1.537,1.537
4,excellent,1.378,1.378,1.378
3,good,1.219,1.219,1.219
2,normal,1.060,1.060,1.060
1,poor,0.848,0.848,0.848
0,awful,0.636,0.636,0.636


In [46]:
# heavy marine armor
expand(src_rating=1.18, stuff=None).sort_values(by="sharp", ascending=False).head(20)

,name,sharp,blunt,heat
6,legendary,2.124,2.124,2.124
5,masterwork,1.711,1.711,1.711
4,excellent,1.534,1.534,1.534
3,good,1.357,1.357,1.357
2,normal,1.180,1.180,1.180
1,poor,0.944,0.944,0.944
0,awful,0.708,0.708,0.708


In [50]:
# plate armor
expand(src_rating=0.73, stuff="metallic").sort_values(by="sharp", ascending=False).head(20)

,name,sharp,blunt,heat
37,legendary_plasteel,1.49796,0.722700,0.854100
39,legendary_uranium,1.41912,0.709560,0.854100
40,legendary_sky steel,1.31400,0.722700,0.459900
41,legendary_chtin,1.20888,0.236520,0.354780
31,masterwork_plasteel,1.20669,0.582175,0.688025
38,legendary_steel,1.18260,0.591300,0.788400
33,masterwork_uranium,1.14318,0.571590,0.688025
25,excellent_plasteel,1.08186,0.521950,0.616850
34,masterwork_sky steel,1.05850,0.582175,0.370475
27,excellent_uranium,1.02492,0.512460,0.616850


In [86]:
with_offsets = armors[~pd.isna(armors.offsets)]
df = with_offsets.copy()
unique_offset_values = []
for offset in with_offsets.offsets:
    for k in offset:
        if k not in unique_offset_values:
            unique_offset_values.append(k)

for offset in unique_offset_values:
    df[offset] = with_offsets.offsets.apply(lambda x: x.get(offset, pd.NA))
    
df.columns

Index(['name', 'stuff_effect_multiplier_armor', 'armor_rating_sharp',
       'armor_rating_blunt', 'armor_rating_heat', 'insulation_cold',
       'insulation_heat', 'stuff_effect_multiplier_insulation_cold',
       'stuff_effect_multiplier_insulation_heat', 'offsets', 'move_speed',
       'SlaveSuppressionOffset', 'MoveSpeed', 'PsychicSensitivity',
       'PsychicEntropyRecoveryRate', 'Flammability', 'ShootingAccuracyPawn',
       'HuntingStealth', 'AimingDelayFactor', 'ToxicSensitivity',
       'PainShockThreshold', 'MentalBreakThreshold', 'MeleeDodgeChance',
       'SocialImpact', 'TradePriceImprovement', 'PlantWorkSpeed',
       'PlantHarvestYield', 'AnimalGatherSpeed', 'AnimalGatherYield',
       'CarryingCapacity', 'CookSpeed', 'Cleanliness', 'ButcheryFleshSpeed',
       'ButcheryFleshEfficiency', 'ConstructionSpeed',
       'ConstructSuccessChance', 'FixBrokenDownBuildingSuccessChance',
       'MedicalSurgerySuccessChance', 'MedicalOperationSpeed',
       'MedicalTendQuality', 'M

In [94]:
df[["name", 'WorkSpeedGlobal', 'GeneralLaborSpeed']]

,name,WorkSpeedGlobal,GeneralLaborSpeed
5,Apparel_Duster,<NA>,<NA>
7,Apparel_PlateArmor,<NA>,<NA>
8,Apparel_FlakVest,<NA>,<NA>
9,Apparel_FlakPants,<NA>,<NA>
10,Apparel_FlakJacket,<NA>,<NA>
0,Apparel_BodyStrap,<NA>,<NA>
1,Apparel_Burka,<NA>,<NA>
0,Apparel_ArmorReconPrestige,<NA>,<NA>
1,Apparel_ArmorHelmetReconPrestige,<NA>,<NA>
2,Apparel_ArmorMarinePrestige,<NA>,<NA>
